<a href="https://colab.research.google.com/github/dansojo/Super-Resolution/blob/main/ESPCN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
import matplotlib.image as img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Lambda
from tensorflow.keras.optimizers import Adam

In [ ]:
def load_images(lr_directory, hr_directory, target_size=(512, 512)):
    x_train = []
    y_train = []

    # 파일 목록을 안전하게 가져오기
    try:
        lr_filenames = sorted(os.listdir(lr_directory))
        hr_filenames = sorted(os.listdir(hr_directory))
    except FileNotFoundError:
        print("Directory not found")
        return [], []

    for lr_file, hr_file in zip(lr_filenames, hr_filenames):
        try:
            lr_image_path = os.path.join(lr_directory, lr_file)
            hr_image_path = os.path.join(hr_directory, hr_file)

            lr_img = Image.open(lr_image_path).resize(target_size, Image.BICUBIC)
            hr_img = Image.open(hr_image_path).resize((2048, 2048), Image.BICUBIC)

            x_train.append(np.array(lr_img))
            y_train.append(np.array(hr_img))
        except (IOError, OSError):
            print(f"Error processing {lr_file} or {hr_file}")

    if len(x_train) > 0 and len(y_train) > 0:
        x_train = np.array(x_train) / 255.0
        y_train = np.array(y_train) / 255.0

    return x_train, y_train

In [ ]:

def ESPCN(scale_factor, num_channels=3):
    # Input layer
    input_layer = Input(shape=(None, None, num_channels), name='input_layer')

    # Feature extraction layer
    features = Conv2D(filters=64, kernel_size=5, padding='same', activation='tanh', name='feature_extraction')(input_layer)

    # Non-linear mapping layer
    mapping = Conv2D(filters=32, kernel_size=3, padding='same', activation='tanh', name='non_linear_mapping')(features)

    # Sub-pixel convolution layer for upscaling
    subpixel = Conv2D(filters=num_channels * (scale_factor ** 2), kernel_size=3, padding='same', activation='tanh', name='subpixel_convolution')(mapping)

    # Rearrange the data in the subpixel layer to form the final high resolution image
    output_layer = Lambda(lambda x: tf.nn.depth_to_space(x, scale_factor), name='pixel_shuffle')(subpixel)

    # Create model
    model = Model(inputs=input_layer, outputs=output_layer, name='ESPCN')
    return model


In [ ]:
# 이미지 데이터셋 로드 경로
dataset_hr_directory = '/content/drive/MyDrive/Super-Resolution/SR_data_set/train/hr'
dataset_lr_directory = '/content/drive/MyDrive/Super-Resolution/SR_data_set/train/lr'
x_train, y_train = load_images(dataset_lr_directory, dataset_hr_directory)

In [ ]:
# Create the espcn_model
scale_factor = 4  # Example scale factor
espcn_model = ESPCN(scale_factor=scale_factor, num_channels=3)
espcn_model.summary()

# Compile the model
espcn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
espcn_model.fit(x_train, y_train, batch_size=16, epochs=100)

espcn_model.save('espcn_model_v1.h5')